
## 1. 모듈 임포트
- tkinter 모듈은 파이썬에서 GUI를 만들기 위해 사용됩니다.
- ttk는 tkinter의 확장 모듈로, 스타일이 개선된 위젯을 제공합니다.
- random 모듈은 무작위 숫자 또는 데이터를 생성하는 데 사용됩니다.

In [30]:
import tkinter as tk  # 기본 tkinter 모듈
from tkinter import ttk  # 확장된 tkinter 위젯 모듈
import random  # 무작위 데이터 생성을 위한 모듈

## 2. 전역 변수 설정
- 게임에서 사용할 주요 변수들을 선언합니다.
- mouse_x, mouse_y는 마우스의 현재 위치를 저장합니다.
- mouse_c는 마우스 클릭 상태를 나타내는 변수입니다.
- cnt는 플레이어가 목적지에 도착한 횟수를 카운트합니다.
- player_speeds, player_stopped, players는 각각 플레이어의 속도, 정지 여부, 생성된 플레이어의 정보를 저장합니다.

In [31]:
# 전역 변수
mouse_x, mouse_y, mouse_c = 0, 0, 0  # 마우스 좌표 및 클릭 상태
cnt = 0  # 플레이어 도착 카운터
player_speeds, player_stopped, players = {}, {}, []  # 플레이어 상태 관리용 딕셔너리

## 마우스 이벤트 처리 함수
- **`mouse_move(e)`**: 마우스의 움직임을 추적하여 `mouse_x`, `mouse_y` 전역 변수에 현재 좌표를 저장하는 함수.
- **`mouse_press(e)`**: 마우스 클릭 이벤트를 처리하여 `mouse_c` 변수를 1로 설정하는 함수.

In [32]:
def mouse_move(e):
    """마우스 움직임 이벤트 처리"""
    global mouse_x, mouse_y
    mouse_x, mouse_y = e.x, e.y

def mouse_press(e):
    """마우스 클릭 이벤트 처리"""
    global mouse_c
    mouse_c = 1

## 윈도우 및 캔버스 설정
- **윈도우 생성 및 설정**: `root` 객체는 tkinter 윈도우를 생성하며, 제목과 크기를 설정합니다.
- **마우스 이벤트 바인딩**: 마우스 움직임과 클릭 이벤트를 각각 `mouse_move`, `mouse_press` 함수와 연결합니다.
- **캔버스 설정**: 1280x720 크기의 캔버스를 생성하고 `root` 윈도우에 추가합니다.

## 초기 페이지 및 전역 변수 설정
- **초기 배경 이미지 설정**: `img` 객체에 초기 배경 이미지를 불러와 중앙에 위치시킵니다.
- **전역 변수 설정**: 현재 페이지 상태 (`index`), 이미지 ID, 이미지 객체 리스트, 이미지 ID 리스트를 초기화합니다.


In [33]:
# 윈도우 및 캔버스 설정
root = tk.Tk()
root.title('노부부와 보호인')
root.resizable(False, False)
root.bind("<Motion>", mouse_move)  # 마우스 움직임 이벤트
root.bind("<ButtonPress>", mouse_press)  # 마우스 클릭 이벤트
root.geometry('1280x720')

canvas = tk.Canvas(root, width=1280, height=720)
canvas.pack()

# 초기 페이지 배경 설정
img = tk.PhotoImage(file='../img/background_pages/start_page.png')
canvas.create_image(640, 360, image=img)

# 전역 변수 설정
index, image_id, img_objects, image_ids = 'start_page', None, [], []
team_info = []
input_texts = ["" for _ in range(7)]
max_teams = 7
input_positions = [100]
text_ids = [None for _ in range(7)]
cursor_id = None
edit_button_ids = [None for _ in range(7)]
edit_mode = [False for _ in range(7)]
input_spacing = 75
active_input = None
input_image_files = [f"../img/icons/input{i}.png" for i in range(2, 8)]
start_button = None
popup_image = None

### 텍스트 입력 및 표시 관련 함수

- 키보드 입력을 처리하는 함수

- 매개변수:
    - event : 키보드 이벤트 객체

- 설명:
현재 활성화된 입력칸에 텍스트를 추가하거나 삭제하고, 
텍스트 디스플레이와 커서 위치를 업데이트합니다.

In [34]:
# 팀 정보 입력 관련 함수
def handle_input(event):
    global active_input
    if active_input is not None:
        if event.keysym == 'BackSpace':
            input_texts[active_input] = input_texts[active_input][:-1]
        elif event.keysym == 'space':
            input_texts[active_input] += ' '
        elif len(event.char) == 1:
            input_texts[active_input] += event.char
        update_text_display(active_input)
        update_cursor_position(active_input)

### 입력칸 텍스트 표시 함수
    
- 특정 입력칸의 텍스트 디스플레이를 업데이트하는 함수

- 매개변수:
    - index : 업데이트할 입력칸의 인덱스

- 설명:
기존 텍스트를 삭제하고 새로운 텍스트를 캔버스에 표시합니다.

In [35]:
def update_text_display(index):
    if text_ids[index]:
        canvas.delete(text_ids[index])
    y_position = input_positions[index]
    text_ids[index] = canvas.create_text(180, y_position + 25, text=input_texts[index], anchor='w', font=('Arial', 12))

### 입력칸 내 커서 표시 함수

- 매개변수:
    - index : 커서를 표시할 입력칸의 인덱스

- 설명:
현재 텍스트의 끝에 커서를 표시하고 깜빡임 효과를 시작합니다.

In [36]:
def update_cursor_position(index):
    global cursor_id
    if cursor_id:
        canvas.delete(cursor_id)
    y_position = input_positions[index]
    text_width = canvas.bbox(text_ids[index])[2] - canvas.bbox(text_ids[index])[0] if text_ids[index] else 0
    cursor_id = canvas.create_line(180 + text_width, y_position + 10, 180 + text_width, y_position + 40, fill='black')
    blink_cursor()

### 커서 깜빡임 효과

- 설명:
    600ms 간격으로 커서를 보이게 하거나 숨깁니다.

In [37]:
def blink_cursor():
    global cursor_id
    if cursor_id:
        current_state = canvas.itemcget(cursor_id, 'state')
        new_state = 'hidden' if current_state == 'normal' else 'normal'
        canvas.itemconfigure(cursor_id, state=new_state)
    if active_input is not None:
        canvas.after(600, blink_cursor)

### 특정 입력칸 활성화

- 특정 입력칸을 활성화하는 함수
    
- 매개변수:
    - index : 활성화할 입력칸의 인덱스

- 설명:
지정된 입력칸을 활성 상태로 만들고 커서를 표시합니다.

In [38]:
def activate_input(index):
    global active_input
    active_input = index
    canvas.focus_set()
    update_cursor_position(index)

### 입력칸 추가 기능

- 새로운 입력칸을 추가하는 함수

- 설명:
최대 팀 수에 도달하지 않았다면 새 입력칸을 추가하고 관련 이미지를 로드합니다.

In [39]:
def add_input_field():
    if len(input_positions) < max_teams:
        new_index = len(input_positions)
        y_position = input_positions[0] + new_index * input_spacing
        input_positions.append(y_position)
        
        try:
            if new_index > 0:
                img_entry = tk.PhotoImage(file=input_image_files[new_index - 1])
                img_objects.append(img_entry)
                image_id = canvas.create_image(635, y_position + 25, image=img_entry)
                image_ids.append(image_id)
            
            input_texts[new_index] = ""
            update_text_display(new_index)
        except tk.TclError as e:
            print(f"이미지 로딩 오류: {e}")

### 수정하기 버튼 기능

- display_edit_button
    -  수정하기 버튼을 표시하는 함수
        
    - 매개변수:
        - index : 수정하기 버튼을 표시할 입력칸의 인덱스

    - 설명:
    지정된 입력칸 옆에 수정하기 버튼 이미지를 로드하고 표시합니다.

- get_button_position
    - 수정하기 버튼의 위치를 계산하는 함수
    
    - 매개변수:
        - index : 버튼 위치를 계산할 입력칸의 인덱스

    - 반환값:
    (x, y) : 버튼의 x, y 좌표

    - 설명:
    첫 번째 버튼은 고정 위치에, 나머지는 입력칸에 맞춰 위치를 계산합니다.

In [40]:
def display_edit_button(index):
    try:
        edit_img = tk.PhotoImage(file='../img/icons/edit_button.png')
        img_objects.append(edit_img)
        button_x, button_y = get_button_position(index)
        edit_button_ids[index] = canvas.create_image(button_x, button_y, image=edit_img)
    except tk.TclError as e:
        print(f"이미지 로딩 오류: {e}")

def get_button_position(index):
    if index == 0:
        return 1070, 130
    else:
        y_position = input_positions[index]
        return 1070, y_position + 25

### 이벤트 처리

- 마우스 클릭 이벤트를 처리하는 함수
    
- 매개변수:
    - event : 마우스 클릭 이벤트 객체

- 설명:
클릭된 위치에 따라 입력칸 활성화, 추가 버튼 처리, 수정 버튼 처리를 수행합니다.

In [41]:
def handle_team_input_page(event):
    global active_input, cursor_id

    if cursor_id:
        canvas.delete(cursor_id)
        cursor_id = None

    for i, y_position in enumerate(input_positions):
        if 160 < event.x < 1000 and y_position < event.y < y_position + 50:
            activate_input(i)
            return
    
    for i, y_position in enumerate(input_positions):
        if 1150 < event.x < 1250 and y_position < event.y < y_position + 50:
            if i >= len(team_info):
                handle_add_button(i)
            else:
                handle_edit_button(i)
            return

    for i in range(len(team_info)):
        button_x, button_y = get_button_position(i)
        if button_x - 40 < event.x < button_x + 40 and button_y - 20 < event.y < button_y + 20:
            handle_edit_button(i)
            return

    active_input = None

### 버튼 클릭 처리

- handle_add_button
    - 추가 버튼 클릭을 처리하는 함수
    
    - 매개변수:
        - index : 클릭된 추가 버튼의 인덱스

    - 설명:
    입력된 팀 정보를 저장하고, 새 입력칸을 추가하며, 수정 버튼을 표시합니다.

- handle_edit_button
    - 수정하기 버튼 클릭을 처리하는 함수
    
    - 매개변수:
        - index : 클릭된 수정하기 버튼의 인덱스

    - 설명:
    입력된 새로운 팀 정보로 기존 정보를 업데이트합니다.

In [42]:
def handle_add_button(index):
    if input_texts[index]:
        team_number = len(team_info) + 1
        team_member = input_texts[index]
        team_info.append({'team_number': team_number, 'team_member': team_member})
        if len(input_positions) < max_teams:
            add_input_field()
        display_edit_button(index)

def handle_edit_button(index):
    if input_texts[index]:
        team_info[index]['team_member'] = input_texts[index]

# 입력 정보 초기화

In [43]:
def reset_team_input():
    global team_info, input_texts, input_positions, text_ids, edit_button_ids, active_input
    team_info = []
    input_texts = ["" for _ in range(7)]
    input_positions = [100]
    text_ids = [None for _ in range(7)]
    edit_button_ids = [None for _ in range(7)]
    active_input = None

## input 페이지 설정

In [44]:
def setup_team_input_page():
    global team_info, input_texts, input_positions, text_ids, edit_button_ids
    team_info = []
    input_texts = ["" for _ in range(7)]
    input_positions = [100]
    text_ids = [None for _ in range(7)]
    edit_button_ids = [None for _ in range(7)]
    add_input_field()

## 아이템 생성 함수
- **함수 설명**: `create_items` 함수는 아이템을 캔버스 위에 랜덤한 위치에 생성하며, 아이템 간의 겹침을 방지합니다.
- **겹침 방지 함수**: `is_overlapping` 함수는 생성된 아이템의 좌표와 비교하여 아이템이 서로 겹치지 않도록 합니다.
- **랜덤 위치 생성**: 각 아이템은 `random.randint`를 사용해 무작위로 생성된 좌표에 배치되며, 위치가 겹치지 않을 때까지 반복합니다.
- **아이템 이미지 로드**: `tk.PhotoImage`를 사용해 이미지를 로드하며, 각 아이템의 정보를 바탕으로 캔버스에 이미지를 추가합니다.
- **리턴 값**: 최종적으로 생성된 아이템 리스트를 반환합니다. 아이템이 성공적으로 생성되지 않을 경우 오류 메시지가 출력됩니다.


In [45]:
def create_items(canvas, item_info, images):
    """아이템을 화면에 생성하는 함수"""
    items = []

    def is_overlapping(x, y, item_size):
        """아이템 간 겹침 방지"""
        for item, _ in items:
            existing_x, existing_y = canvas.coords(item)
            min_dist_x, min_dist_y = item_size[0] + 100, item_size[1] + 100
            if (existing_x - min_dist_x < x < existing_x + min_dist_x and
                existing_y - min_dist_y < y < existing_y + min_dist_y):
                return True
        return False

    for item_type, info in item_info.items():
        item_photo = tk.PhotoImage(file=info['path'])
        images.append(item_photo)
        item_size = (item_photo.width() // 2, item_photo.height() // 2)

        for _ in range(info["count"]):
            while True:
                x, y = random.randint(100, 928), random.randint(50, 670)  # 랜덤 위치 생성
                if not is_overlapping(x, y, item_size):
                    break

            item = canvas.create_image(x, y, image=item_photo)
            if item:
                items.append((item, item_type))
            else:
                print(f"{item_type} 생성 실패")
    
    return items

## 게임 창 생성 및 초기화
- 이 함수는 게임의 주요 요소를 초기화하고 게임 창을 설정합니다. 주요 기능은 다음과 같습니다:
    - 아이템 정보 설정 및 생성
    - 플레이어 이미지 로드
    - 팀 수에 따른 플레이어 배치 계산
    - 플레이어 캐릭터 생성 및 배치
    - 게임 시작 버튼 생성
- 이 함수는 사용자가 입력한 팀 정보에 따라 동적으로 플레이어 수를 조정하고, 화면 중앙을 기준으로 균등한 간격으로 플레이어들을 배치합니다. 또한, 게임에 필요한 아이템들을 생성하고 시작 버튼을 배치하여 게임 준비를 완료합니다.


In [46]:
def create_game_window():
    global players, start_button
    players = []
    item_info = {
    "chestnut": {"path": "../img/items/chestnut.png", "count": 3},
    "cobweb": {"path": "../img/items/cobweb.png", "count": 3},
    "poison_mushroom": {"path": "../img/items/poison_mushroom.png", "count": 3},
    "balloon": {"path": "../img/items/balloon.png", "count": 3},
    "gold_mushroom": {"path": "../img/items/gold_mushroom.png", "count": 3},
    "hole": {"path": "../img/items/hole.png", "count": 3},
    "random_box": {"path": "../img/items/random_box.png", "count": 3},
    }

    root.images = []  # 이미지 참조를 유지하여 삭제 방지
    items = create_items(canvas, item_info, root.images)
    player_images = [
        tk.PhotoImage(file=f'../img/players/player{i}.png') for i in range(len(team_info))
    ]
    root.images.extend(player_images)

    num_players = len(team_info)
    canvas_height = 720  # 캔버스의 높이
    total_spacing = 600  # 플레이어들이 차지할 전체 세로 공간
    
    if num_players > 1:
        spacing = total_spacing / (num_players - 1)  # 플레이어 간 간격
    else:
        spacing = 0  # 플레이어가 1명일 경우 간격은 의미 없음
    
    start_y = (canvas_height - total_spacing) / 2  # 시작 y 좌표 (중앙 정렬)

    for i, team in enumerate(team_info):
        y_position = start_y + i * spacing
        player = canvas.create_image(24, y_position, image=player_images[i])
        players.append(player)


    # 기존 버튼이 있다면 삭제
    if start_button is not None:
        start_button.destroy()

    start_button = ttk.Button(root, text="Start", command=lambda: start_race(canvas, players, items, start_button))
    start_button.place(x=1100, y=600)
    return players, items

## 게임 시작 함수
- **함수 설명**: `start_race` 함수는 게임을 시작하고 각 플레이어의 이동을 제어합니다.
- **스타트 버튼 비활성화**: 버튼이 클릭되면 더 이상 클릭할 수 없도록 비활성화합니다.
- **플레이어 이동 시작**: 각 플레이어에 대해 `canvas.after`를 사용하여 지연된 시간(100ms) 후에 `move_player` 함수를 호출합니다. 이때, 람다 함수를 사용하여 현재 플레이어를 매개변수로 전달합니다.
- **아이템과 버튼**: 플레이어는 주어진 아이템을 피하거나 수집하며 이동하게 됩니다. 


In [47]:
def start_race(canvas, players, items, start_button):
    """게임 시작: 플레이어가 이동을 시작"""
    start_button.config(state='disabled')
    for player in players:
        canvas.after(100, lambda p=player: move_player(canvas, p, items))

## 플레이어 이동 함수
- **함수 설명**: `move_player` 함수는 각 플레이어의 움직임을 제어합니다.
- **플레이어 속도 및 정지 상태 초기화**: `player_speeds`와 `player_stopped` 딕셔너리를 사용하여 각 플레이어의 기본 속도와 정지 상태를 설정합니다.
- **정지 상태 처리**: 만약 플레이어가 정지 상태라면, 100ms 후에 다시 `move_player` 함수를 호출하여 계속 상태를 확인합니다.
- **플레이어 이동**: `dx`와 `dy`를 계산하여 플레이어를 이동시키며, 화면 경계를 넘지 않도록 조정합니다.
- **아이템 충돌 처리**: 모든 아이템과의 충돌을 검사하여, 충돌이 발생할 경우 `handle_item_collision` 함수를 호출하여 적절한 처리를 합니다.
- **완주 체크**: 플레이어가 경주를 완주했는지 확인하고, 그렇지 않다면 계속해서 이동을 반복합니다.


In [48]:
def move_player(canvas, player, items):
    """플레이어가 움직이는 함수"""
    player_speeds.setdefault(player, 5)  # 기본 속도 5
    player_stopped.setdefault(player, False)  # 기본 정지 상태 False

    if player_stopped[player]:  # 플레이어가 정지 상태일 때
        canvas.after(100, lambda: move_player(canvas, player, items))
        return

    dx, dy = player_speeds[player], random.choice([-2, 0, 2]) * player_speeds[player]
    canvas.move(player, dx, dy)  # 플레이어 이동

    player_coords = canvas.coords(player)
    # 화면 밖으로 나가는 것 방지
    player_height = 50  # 플레이어 이미지의 대략적인 높이
    if player_coords[1] < 20:
        canvas.move(player, 0, 20 - player_coords[1])
    elif player_coords[1] > 720 - player_height:
        canvas.move(player, 0, (720 - player_height) - player_coords[1])

    # 아이템 충돌 처리
    for item, item_type in items[:]:
        if check_collision(player_coords, canvas.coords(item)):
            handle_item_collision(canvas, player, item_type, item, items)

    if not check_finish(canvas, player):  # 완주 체크
        canvas.after(100, lambda: move_player(canvas, player, items))


## 아이템 충돌 처리 함수
- **함수 설명**: `handle_item_collision` 함수는 플레이어가 특정 아이템과 충돌했을 때 발생하는 행동을 정의합니다.
- **아이템 종류별 행동**:
  - **"chestnut"**: 플레이어를 왼쪽으로 100픽셀 이동시킵니다.
  - **"cobweb"**: 플레이어를 정지 상태로 만들고, 3초 후에 `resume_player` 함수를 호출하여 다시 움직일 수 있도록 합니다.
  - **"ballon"**: 플레이어를 오른쪽으로 100픽셀 이동시킵니다.
  - **"gold"**: 플레이어의 속도를 5로 증가시킵니다.
  - **"hole"**: 플레이어를 시작 위치로 되돌립니다.
  - **"poison"**: 플레이어의 속도를 3으로 감소시킵니다.
  - **"random"**: `random_action` 함수를 호출하여 랜덤한 행동을 수행합니다.
- **아이템 삭제**: 충돌한 아이템을 캔버스에서 삭제하고, `items` 리스트에서 제거합니다.


In [49]:
def handle_item_collision(canvas, player, item_type, item, items):
    global player_speeds  # 기존 속도를 저장하고 복구하기 위해 필요

    """아이템과 충돌 시 아이템 별 행동 처리"""
    if item_type == "chestnut":
        canvas.move(player, -100, 0)
    elif item_type == "cobweb":
        player_stopped[player] = True
        canvas.after(3000, lambda: resume_player(player))
    elif item_type == "balloon":
        canvas.move(player, 100, 0)
    elif item_type == "gold_mushroom":
        original_speed = player_speeds[player]  # 현재 속도 저장
        player_speeds[player] = 7
        canvas.after(2000, lambda: restore_speed(player, original_speed))  # 2초 후 원래 속도로 복구
    elif item_type == "hole":
        canvas.move(player, -canvas.coords(player)[0] + 24, 0)
    elif item_type == "poison_mushroom":
        original_speed = player_speeds[player]  # 현재 속도 저장
        player_speeds[player] = 3
        canvas.after(2000, lambda: restore_speed(player, original_speed))  # 2초 후 원래 속도로 복구
    elif item_type == "random_box":
        random_action(canvas)

    canvas.delete(item)
    items.remove((item, item_type))

## 속도 원복 함수

In [50]:
def restore_speed(player, original_speed):
    """플레이어의 속도를 원래대로 복구하는 함수"""
    global player_speeds
    player_speeds[player] = original_speed

## 플레이어 재개 함수
- **함수 설명**: `resume_player` 함수는 정지된 플레이어가 다시 이동할 수 있도록 설정합니다.
- **기능**: 
  - 주어진 플레이어의 `player_stopped` 상태를 `False`로 변경하여, 플레이어가 다시 움직일 수 있게 만듭니다.


In [51]:
def resume_player(player):
    """정지된 플레이어 다시 이동 가능하게 설정"""
    player_stopped[player] = False

## 랜덤 행동 처리 함수
- **함수 설명**: `random_action` 함수는 플레이어들이 수행할 랜덤한 행동을 처리합니다.
- **기능**:
  - **`all_to_start`**: 모든 플레이어를 시작 위치로 되돌립니다.
  - **`shuffle_players`**: 플레이어들의 위치를 무작위로 섞어 재배치합니다.


In [52]:
def random_action(canvas):
    """랜덤 행동 처리"""
    global popup_image
    action = random.choice(["all_to_start", "shuffle_players"])

    # 게임 일시 정지
    global player_stopped
    for player in players:
        player_stopped[player] = True

    # 팝업 이미지 표시
    popup_image = tk.PhotoImage(file=f'../img/icons/{action}.png')  # 팝업 이미지 경로
    popup = canvas.create_image(500, 360, image=popup_image)  # 화면 중앙에 표시
    canvas.tag_raise("popup")  # 팝업을 최상위 레이어로 올림

    def remove_popup_and_resume():
        # 팝업 제거
        canvas.delete(popup)

        if action == "all_to_start":
            for player in players:
                if canvas.coords(player)[0] < 1008:  # 도착선(1008) 이전의 플레이어만
                    canvas.coords(player, 24, canvas.coords(player)[1])
        
        elif action == "shuffle_players":
            active_players = []
            active_positions = []
            
            for player in players:
                if canvas.coords(player)[0] < 1008:
                    active_players.append(player)
                    active_positions.append(canvas.coords(player))
            
            if active_positions:
                random.shuffle(active_positions)
                
                for i, player in enumerate(active_players):
                    canvas.coords(player, *active_positions[i])

        # 게임 재개
        for player in players:
            player_stopped[player] = False
            canvas.after(100, lambda p=player: move_player(canvas, p, items))

    # 2초 후 팝업 제거 및 게임 재개
    canvas.after(2000, remove_popup_and_resume)

## 충돌 확인 함수
- **함수 설명**: `check_collision` 함수는 플레이어와 아이템 간의 충돌 여부를 확인합니다.
- **매개변수**:
  - `player_coords`: 플레이어의 현재 좌표 (x, y).
  - `item_coords`: 아이템의 현재 좌표 (x, y).
- **반환값**: 플레이어와 아이템 간의 충돌 여부를 나타내는 불리언 값.
- **구현 방식**:
  - 플레이어와 아이템의 중심 좌표를 계산합니다.
  - 두 점 사이의 거리를 구하고, 두 객체의 반지름을 합한 값과 비교하여 충돌 여부를 판단합니다.


In [53]:
def check_collision(player_coords, item_coords):
    """플레이어와 아이템 간 충돌 여부 확인"""
    player_center_x, player_center_y = player_coords[0] + 25, player_coords[1] + 25
    item_center_x, item_center_y = item_coords[0] + 17.5, item_coords[1] + 17.5
    distance = ((player_center_x - item_center_x) ** 2 + (player_center_y - item_center_y) ** 2) ** 0.5
    return distance <= (45 / 2) + (30 / 2)

## 완주 확인 함수
- **함수 설명**: `check_finish` 함수는 플레이어가 경주를 완주했는지 확인합니다.
- **매개변수**:
  - `canvas`: 게임의 캔버스 객체.
  - `player`: 현재 확인할 플레이어의 ID.
  - `start_button`: 게임 시작 버튼 객체.
- **전역 변수**: 
  - `cnt`: 현재까지 완주한 플레이어 수를 세기 위한 변수.
  - `index`: 현재 페이지 상태를 나타내는 변수.
  - `image_id`: 현재 표시된 배경 이미지의 ID.
- **구현 방식**:
  - 플레이어의 x좌표가 1008 이상인지 확인합니다.
  - 모든 플레이어가 완주했을 경우, 게임 종료 화면으로 전환하고 배경 이미지를 변경합니다.
  - 완주 여부에 따라 True 또는 False를 반환합니다.


In [54]:
def check_finish(canvas, player):
    """플레이어가 완주했는지 확인"""
    global cnt, index, image_id, players, items, start_button

    if canvas.coords(player)[0] >= 1008:
        cnt += 1
        if cnt == len(players):  # 모든 플레이어가 완주했을 때
            # 모든 요소 삭제
            canvas.delete("all")
            
            # 플레이어와 아이템 리스트 초기화
            players = []
            items = []

            # 시작 버튼 제거
            if start_button is not None:
                start_button.destroy()
                start_button = None
            
            # 새로운 엔딩 이미지 로드 및 표시
            img = tk.PhotoImage(file='../img/background_pages/end_page1.png')
            img_objects.append(img)
            image_id = canvas.create_image(640, 360, image=img)
            
            # 게임 상태 초기화
            index = 'end_page1'
            cnt = 0  # 카운터 리셋
            
            # 게임 메인 함수 호출
            root.after(100, game_main)  # 약간의 지연을 두고 game_main 호출
            
        return True
    return False

## 이미지 로드 함수
- **함수 설명**: `load_image` 함수는 지정된 경로에서 이미지를 로드하여 반환합니다.
- **매개변수**:
  - `file_path`: 로드할 이미지 파일의 경로.
- **전역 변수**:
  - `img_objects`: 이미지 객체를 저장하기 위한 리스트. 이미지가 가비지 컬렉션에 의해 삭제되지 않도록 유지합니다.
- **구현 방식**:
  - `tk.PhotoImage`를 사용하여 이미지를 로드하고, 로드된 이미지를 `img_objects` 리스트에 추가합니다.
  - 최종적으로 로드된 이미지를 반환합니다.


In [55]:
def load_image(file_path):
    """이미지 로드 함수"""
    img = tk.PhotoImage(file=file_path)
    img_objects.append(img)
    return img

## 페이지 전환 처리 함수
- **함수 설명**: `switch_page` 함수는 새로운 페이지로의 전환을 처리합니다.
- **매개변수**:
  - `new_page`: 전환할 새로운 페이지의 이름.
  - `image_path`: 새로 표시할 이미지의 파일 경로.
- **전역 변수**:
  - `image_id`: 현재 표시되고 있는 이미지의 ID. 이전 이미지를 삭제하는 데 사용됩니다.
  - `index`: 현재 페이지 상태를 저장하는 변수.
- **구현 방식**:
  - `load_image` 함수를 호출하여 새로운 이미지를 로드합니다.
  - 현재 이미지가 존재할 경우, `canvas.delete`를 사용하여 해당 이미지를 삭제합니다.
  - 새로운 이미지를 캔버스에 생성하고, `image_id`에 저장합니다.
  - `index` 변수를 업데이트하여 현재 페이지 상태를 반영합니다.


In [56]:
def switch_page(new_page, image_path):
    """페이지 전환을 처리하는 함수"""
    global image_id
    img = load_image(image_path)
    if image_id:  # 이전 이미지 삭제
        canvas.delete(image_id)
    image_id = canvas.create_image(640, 360, image=img)  # 새로운 이미지 생성
    global index
    index = new_page  # 페이지 상태 업데이트


## 마우스 클릭 영역 확인 함수
- **함수 설명**: `handle_click` 함수는 마우스 클릭이 특정 영역 내에서 발생했는지 확인합니다.
- **매개변수**:
  - `x1`: 클릭 영역의 왼쪽 경계 x 좌표.
  - `y1`: 클릭 영역의 위쪽 경계 y 좌표.
  - `x2`: 클릭 영역의 오른쪽 경계 x 좌표.
  - `y2`: 클릭 영역의 아래쪽 경계 y 좌표.
- **반환값**: 
  - 클릭이 영역 내에서 발생하면 `True`를 반환하고, 그렇지 않으면 `False`를 반환합니다.
- **구현 방식**:
  - 현재 마우스의 x 좌표(`mouse_x`)와 y 좌표(`mouse_y`)가 주어진 영역의 경계 내에 있는지 비교하여 클릭 여부를 판단합니다.


In [57]:
def handle_click(x1, y1, x2, y2):
    """마우스 클릭 영역을 확인하는 함수"""
    return x1 < mouse_x < x2 and y1 < mouse_y < y2

## 게임 메인 루프 함수
- **함수 설명**: `game_main` 함수는 게임의 다양한 페이지 상태를 관리하고, 마우스 클릭 이벤트에 따라 페이지를 전환합니다.
- **전역 변수**:
  - `mouse_c`: 마우스 클릭 상태.
  - `index`: 현재 페이지 상태를 나타내는 변수.
  - `image_id`: 현재 표시된 이미지의 ID.
  - `image_ids`: 생성된 이미지의 ID 리스트.
  - `cnt`: 완주한 플레이어 수.

- **상태별 처리**:
  - **intro_page**: 
    - 클릭 시 다른 페이지로 전환합니다.
  - **dc_page1** ~ **dc_page4**: 
    - 각 페이지에서 다음 페이지로 이동하거나 이전 페이지로 돌아갈 수 있습니다.
  - **info_page**: 
    - 클릭한 위치에 정보를 표시하고, 특정 클릭 시 페이지 전환.
  - **end_page1** 및 **end_page2**: 
    - 종료 페이지에서 다양한 버튼 클릭에 따라 페이지 전환을 처리합니다.

- **주요 로직**:
  - 클릭 이벤트가 발생하면 해당 영역에 따라 페이지를 전환하고, 마우스 클릭 상태를 초기화합니다.
  - 마지막에, 현재 페이지가 `main_page`가 아닐 경우 100ms 후에 `game_main` 함수를 재호출하여 지속적으로 업데이트합니다.
  - 만약 현재 페이지가 `main_page`라면 `create_game_window` 함수를 호출하여 게임을 시작합니다.


In [58]:
def game_main():
    global mouse_c, index, image_id, image_ids, cnt
    cnt = 0

    if index == 'start_page':
        if mouse_c == 1:
            if handle_click(300, 580, 550, 690):
                switch_page('description_1', '../img/background_pages/description_1.png')
            if handle_click(730, 580, 980, 690):
                switch_page('team_input', '../img/background_pages/team_input.png')
                reset_team_input()
            mouse_c = 0

    elif index == 'description_1':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_2', '../img/background_pages/description_2.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('start_page', '../img/background_pages/start_page.png')
            mouse_c = 0

    elif index == 'description_2':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_3', '../img/background_pages/description_3.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('description_1', '../img/background_pages/description_1.png')
            mouse_c = 0

    elif index == 'description_3':
        if mouse_c == 1:
            if handle_click(1080, 150, 1180, 250):
                switch_page('description_item', '../img/background_pages/description_item.png')
            elif handle_click(100, 150, 200, 250):
                switch_page('description_2', '../img/background_pages/description_2.png')
            mouse_c = 0

    elif index == 'description_item':
        if mouse_c == 1:
            if handle_click(540, 600, 740, 700):
                switch_page('start_page', '../img/background_pages/start_page.png')
                mouse_c = 0

    elif index == 'team_input':
        canvas.bind("<Button-1>", handle_team_input_page)
        root.bind("<Key>", handle_input)
        if mouse_c == 1:
            if handle_click(1050, 620, 1210, 700):
                switch_page('game_page', '../img/background_pages/game_page.png')
                canvas.unbind("<Button-1>")
                root.unbind("<Key>")
                create_game_window()
            mouse_c = 0

    elif index == 'end_page1':
        if mouse_c == 1:
            if handle_click(30, 400, 330, 550):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 175, 840, 205):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 240, 840, 270):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 305, 840, 335):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 370, 840, 400):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 435, 840, 465):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 500, 840, 530):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(800, 565, 840, 595):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(30, 550, 330, 700):
                switch_page('start_page', '../img/background_pages/start_page.png')
            mouse_c = 0

    elif index == 'end_page2':
        if mouse_c == 1:
            if handle_click(30, 550, 330, 700):
                switch_page('start_page', '../img/background_pages/start_page.png')
            elif handle_click(770, 175, 840, 205):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 240, 840, 270):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 305, 840, 335):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 370, 840, 400):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 435, 840, 465):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 500, 840, 530):
                switch_page('game_page', '../img/background_pages/game_page.png')
            elif handle_click(770, 565, 840, 595):
                switch_page('game_page', '../img/background_pages/game_page.png')
            mouse_c = 0

    if index != 'game_page':
        root.after(100, game_main)  # 100ms 후에 다시 호출
    else:
        create_game_window()
        

# 게임 시작
game_main()

# Tkinter 메인 루프 실행
root.mainloop()
